***
# Tournament Seeding: NCAA Men's March Madness
***

## Imports & Global Settings

In [552]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from Utils import *

In [553]:
pd.set_option('display.max_columns', None)
jimmer = 32 # Jimmer Fredette jersey number for randomizaion
duke = 1181 # TeamID for Duke to use for reference / debugging

***
## Tournament Seeding
***

In [554]:
# Load in Tournament Info
teams = pd.read_csv('./Data/Mteams.csv', dtype={'TeamID': str}).set_index('TeamID')['TeamName']
tournament_seeds = pd.read_csv('./Data/MNCAATourneySeeds.csv', dtype={'TeamID': str})
display(tournament_seeds)

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374
...,...,...,...
2621,2025,Z12,1161
2622,2025,Z13,1213
2623,2025,Z14,1423
2624,2025,Z15,1303


In [555]:
tournament_seeds['TeamName'] = tournament_seeds['TeamID'].map(teams)
display(tournament_seeds)

,Season,Seed,TeamID,TeamName
0,1985,W01,1207,Georgetown
1,1985,W02,1210,Georgia Tech
2,1985,W03,1228,Illinois
3,1985,W04,1260,Loyola-Chicago
4,1985,W05,1374,SMU
...,...,...,...,...
2621,2025,Z12,1161,Colorado St
2622,2025,Z13,1213,Grand Canyon
2623,2025,Z14,1423,UNC Wilmington
2624,2025,Z15,1303,NE Omaha


In [556]:
print(tournament_seeds['Seed'].sample(5))
print("--------------------------")
print(tournament_seeds['Seed'].str[0].unique())

2124     Y09
1966     X04
2008    Z11b
2394     Y07
1998     Z02
Name: Seed, dtype: object
--------------------------
['W' 'X' 'Y' 'Z']


As part of the seeding process and bracket formation, the tournament field is divided into four (4) regions—`W`, `X`, `Y`, and `Z`—each containing seeds **1 through 16** (e.g., `W01` = #1 Seed in `RegionW`; `Z12` = #12 seed in `RegionZ`, etc.).


Whichever region’s name comes first alphabetically is assigned to `RegionW`. The region that plays against `RegionW` in the national semifinals (*The Final Four*)—or is positioned on the **same side of the bracket**—is assigned to `RegionX`.  

For the other two regions, the one that comes first alphabetically is assigned to `RegionY`, and the remaining region is assigned to `RegionZ`.  

This standardized naming ensures consistency, even if the official region names change from year to year.

In [557]:
tournament_seeds['Region'] = tournament_seeds['Seed'].str[0]
tournament_seeds['SeedNum'] = tournament_seeds['Seed'].str[1:]
tournament_seeds

,Season,Seed,TeamID,TeamName,Region,SeedNum
0,1985,W01,1207,Georgetown,W,01
1,1985,W02,1210,Georgia Tech,W,02
2,1985,W03,1228,Illinois,W,03
3,1985,W04,1260,Loyola-Chicago,W,04
4,1985,W05,1374,SMU,W,05
...,...,...,...,...,...,...
2621,2025,Z12,1161,Colorado St,Z,12
2622,2025,Z13,1213,Grand Canyon,Z,13
2623,2025,Z14,1423,UNC Wilmington,Z,14
2624,2025,Z15,1303,NE Omaha,Z,15


In [558]:
tournament_seeds['SeedNum'].unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '14', '15', '16', '16a', '16b', '12a', '12b', '11a',
       '11b', '14a', '14b', '13a', '13b', '10a', '10b'], dtype=object)

In [559]:
tournament_seeds[tournament_seeds['SeedNum'].str.contains('a|b')]

,Season,Seed,TeamID,TeamName,Region,SeedNum
1071,2001,Y16a,1322,Northwestern LA,Y,16a
1072,2001,Y16b,1457,Winthrop,Y,16b
1104,2002,W16a,1108,Alcorn St,W,16a
1105,2002,W16b,1373,Siena,W,16b
1185,2003,X16a,1411,TX Southern,X,16a
...,...,...,...,...,...,...
2586,2025,X11b,1462,Xavier,X,11b
2602,2025,Y11a,1314,North Carolina,Y,11a
2603,2025,Y11b,1361,San Diego St,Y,11b
2608,2025,Y16a,1106,Alabama St,Y,16a


### **Play-In Games & Their Impact on Seeding** 
***

In **2011**, the NCAA introduced the **First Four** format, expanding the tournament to **68 teams**. This created **four play-in games**, where teams compete for a spot in the **64-team bracket**.

The **First Four** consists of:
- **Two games between the lowest-seeded automatic qualifiers**, playing for a **16 seed**.
- **Two games between the lowest-seeded at-large teams**, competing for a **higher seed** (e.g., an 11 or 12 seed).  

***

### **Automatic Qualifiers vs. At-Large Teams:**
- **Automatic Qualifiers** – Teams that earn a bid to the NCAA Tournament by **winning their conference tournament**. Each of the **32 Division I conferences** receives **one automatic bid**.
- **At-Large Teams** – Teams that did ****NOT**** win their conference tournament but are selected by the **NCAA Selection Committee** based on their overall performance. The remaining **36 spots** in the tournament are filled by at-large teams.

Since play-in teams share the **same numerical seed**, an **additional letter (`a` or `b`)** is assigned to differentiate them:
- The **"a" team** is assigned to the school with the **lower Team ID**.
- The **"b" team** is assigned to the other play-in team at the same seed level.

This ensures that **each team has a unique identifier** while maintaining the traditional seeding structure, but for the purposes of our model, the additional letters can be dropped.

In [560]:
tournament_seeds['SeedNum'] = tournament_seeds['SeedNum'].str.replace('a', '').str.replace('b', '').astype(int)
tournament_seeds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2626 entries, 0 to 2625
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Season    2626 non-null   int64 
 1   Seed      2626 non-null   object
 2   TeamID    2626 non-null   object
 3   TeamName  2626 non-null   object
 4   Region    2626 non-null   object
 5   SeedNum   2626 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 123.2+ KB


In [561]:
# Load in data from Data_Collection notebook
regular_season_stats = pd.read_csv('./Data/Live/RegularSeasonStats.csv', dtype={'TeamID': str})

In [562]:
# Merge Seed data onto regular season stats
regular_season_stats = regular_season_stats.merge(
    tournament_seeds[['Season', 'TeamID', 'Seed', 'Region', 'SeedNum']],
    on=['Season', 'TeamID'],
    how='left'
)

In [563]:
regular_season_stats.head()

,Season,TeamID,TeamName,ConfAbbrev,Description,TotalGames,TotalWins,TotalLosses,WinningPercentage,Score_PerGame,OppScore_PerGame,NumOT_PerGame,FGM_PerGame,FGA_PerGame,FGM3_PerGame,FGA3_PerGame,FTM_PerGame,FTA_PerGame,OR_PerGame,DR_PerGame,Ast_PerGame,TO_PerGame,Stl_PerGame,Blk_PerGame,PF_PerGame,OppFGM_PerGame,OppFGA_PerGame,OppFGM3_PerGame,OppFGA3_PerGame,OppFTM_PerGame,OppFTA_PerGame,OppOR_PerGame,OppDR_PerGame,OppAst_PerGame,OppTO_PerGame,OppStl_PerGame,OppBlk_PerGame,OppPF_PerGame,Win_PerGame,FG_Percentage_PerGame,FG3_Percentage_PerGame,FT_Percentage_PerGame,Ast_TO_Ratio_PerGame,OppFG_Percentage_PerGame,OppFG3_Percentage_PerGame,OppFT_Percentage_PerGame,OppAst_TO_Ratio_PerGame,Poss_PerGame,OppPoss_PerGame,OffEff_PerGame,DefEff_PerGame,OppOffEff_PerGame,OppDefEff_PerGame,Score_Diff_PerGame,FGM_Diff_PerGame,FGA_Diff_PerGame,FGM3_Diff_PerGame,FGA3_Diff_PerGame,FTM_Diff_PerGame,FTA_Diff_PerGame,OR_Diff_PerGame,DR_Diff_PerGame,Ast_Diff_PerGame,TO_Diff_PerGame,Stl_Diff_PerGame,Blk_Diff_PerGame,PF_Diff_PerGame,Poss_Diff_PerGame,OffEff_Diff_PerGame,DefEff_Diff_PerGame,Score_Diff_Pct_PerGame,FGM_Diff_Pct_PerGame,FGA_Diff_Pct_PerGame,FGM3_Diff_Pct_PerGame,FGA3_Diff_Pct_PerGame,FTM_Diff_Pct_PerGame,FTA_Diff_Pct_PerGame,OR_Diff_Pct_PerGame,DR_Diff_Pct_PerGame,Ast_Diff_Pct_PerGame,TO_Diff_Pct_PerGame,Stl_Diff_Pct_PerGame,Blk_Diff_Pct_PerGame,PF_Diff_Pct_PerGame,Poss_Diff_Pct_PerGame,OffEff_Diff_Pct_PerGame,DefEff_Diff_Pct_PerGame,Score_Diff_Opp_PerGame,FGM_Diff_Opp_PerGame,FGA_Diff_Opp_PerGame,FGM3_Diff_Opp_PerGame,FGA3_Diff_Opp_PerGame,FTM_Diff_Opp_PerGame,FTA_Diff_Opp_PerGame,OR_Diff_Opp_PerGame,DR_Diff_Opp_PerGame,Ast_Diff_Opp_PerGame,TO_Diff_Opp_PerGame,Stl_Diff_Opp_PerGame,Blk_Diff_Opp_PerGame,PF_Diff_Opp_PerGame,Poss_Diff_Opp_PerGame,OffEff_Diff_Opp_PerGame,DefEff_Diff_Opp_PerGame,Score_Diff_Pct_Opp_PerGame,FGM_Diff_Pct_Opp_PerGame,FGA_Diff_Pct_Opp_PerGame,FGM3_Diff_Pct_Opp_PerGame,FGA3_Diff_Pct_Opp_PerGame,FTM_Diff_Pct_Opp_PerGame,FTA_Diff_Pct_Opp_PerGame,OR_Diff_Pct_Opp_PerGame,DR_Diff_Pct_Opp_PerGame,Ast_Diff_Pct_Opp_PerGame,TO_Diff_Pct_Opp_PerGame,Stl_Diff_Pct_Opp_PerGame,Blk_Diff_Pct_Opp_PerGame,PF_Diff_Pct_Opp_PerGame,Poss_Diff_Pct_Opp_PerGame,OffEff_Diff_Pct_Opp_PerGame,DefEff_Diff_Pct_Opp_PerGame,KenPomRank_PerGame,OppKenPomRank_PerGame,KenPomDiff_PerGame,OppKenPomDiff_PerGame,Score_PerWin,OppScore_PerWin,NumOT_PerWin,FGM_PerWin,FGA_PerWin,FGM3_PerWin,FGA3_PerWin,FTM_PerWin,FTA_PerWin,OR_PerWin,DR_PerWin,Ast_PerWin,TO_PerWin,Stl_PerWin,Blk_PerWin,PF_PerWin,OppFGM_PerWin,OppFGA_PerWin,OppFGM3_PerWin,OppFGA3_PerWin,OppFTM_PerWin,OppFTA_PerWin,OppOR_PerWin,OppDR_PerWin,OppAst_PerWin,OppTO_PerWin,OppStl_PerWin,OppBlk_PerWin,OppPF_PerWin,Win_PerWin,FG_Percentage_PerWin,FG3_Percentage_PerWin,FT_Percentage_PerWin,Ast_TO_Ratio_PerWin,OppFG_Percentage_PerWin,OppFG3_Percentage_PerWin,OppFT_Percentage_PerWin,OppAst_TO_Ratio_PerWin,Poss_PerWin,OppPoss_PerWin,OffEff_PerWin,DefEff_PerWin,OppOffEff_PerWin,OppDefEff_PerWin,Score_Diff_PerWin,FGM_Diff_PerWin,FGA_Diff_PerWin,FGM3_Diff_PerWin,FGA3_Diff_PerWin,FTM_Diff_PerWin,FTA_Diff_PerWin,OR_Diff_PerWin,DR_Diff_PerWin,Ast_Diff_PerWin,TO_Diff_PerWin,Stl_Diff_PerWin,Blk_Diff_PerWin,PF_Diff_PerWin,Poss_Diff_PerWin,OffEff_Diff_PerWin,DefEff_Diff_PerWin,Score_Diff_Pct_PerWin,FGM_Diff_Pct_PerWin,FGA_Diff_Pct_PerWin,FGM3_Diff_Pct_PerWin,FGA3_Diff_Pct_PerWin,FTM_Diff_Pct_PerWin,FTA_Diff_Pct_PerWin,OR_Diff_Pct_PerWin,DR_Diff_Pct_PerWin,Ast_Diff_Pct_PerWin,TO_Diff_Pct_PerWin,Stl_Diff_Pct_PerWin,Blk_Diff_Pct_PerWin,PF_Diff_Pct_PerWin,Poss_Diff_Pct_PerWin,OffEff_Diff_Pct_PerWin,DefEff_Diff_Pct_PerWin,Score_Diff_Opp_PerWin,FGM_Diff_Opp_PerWin,FGA_Diff_Opp_PerWin,FGM3_Diff_Opp_PerWin,FGA3_Diff_Opp_PerWin,FTM_Diff_Opp_PerWin,FTA_Diff_Opp_PerWin,OR_Diff_Opp_PerWin,DR_Diff_Opp_PerWin,Ast_Diff_Opp_PerWin,TO_Diff_Opp_PerWin,Stl_Diff_Opp_PerWin,Blk_Diff_Opp_PerWin,PF_Diff_Opp_PerWin,Poss_Diff_Opp_PerWin,OffEff_Diff_Opp_PerWin,DefEff_Diff_Opp_PerWin,Score_Diff_Pct_Opp_PerWin,FGM_Diff_Pct_Opp_PerWin,FGA_Diff_Pc

Since only 64 teams make the tournament, there will be a large number of missing values for those teams that did ***not*** make the bracket.

In [564]:
regular_season_stats.tail()

,Season,TeamID,TeamName,ConfAbbrev,Description,TotalGames,TotalWins,TotalLosses,WinningPercentage,Score_PerGame,OppScore_PerGame,NumOT_PerGame,FGM_PerGame,FGA_PerGame,FGM3_PerGame,FGA3_PerGame,FTM_PerGame,FTA_PerGame,OR_PerGame,DR_PerGame,Ast_PerGame,TO_PerGame,Stl_PerGame,Blk_PerGame,PF_PerGame,OppFGM_PerGame,OppFGA_PerGame,OppFGM3_PerGame,OppFGA3_PerGame,OppFTM_PerGame,OppFTA_PerGame,OppOR_PerGame,OppDR_PerGame,OppAst_PerGame,OppTO_PerGame,OppStl_PerGame,OppBlk_PerGame,OppPF_PerGame,Win_PerGame,FG_Percentage_PerGame,FG3_Percentage_PerGame,FT_Percentage_PerGame,Ast_TO_Ratio_PerGame,OppFG_Percentage_PerGame,OppFG3_Percentage_PerGame,OppFT_Percentage_PerGame,OppAst_TO_Ratio_PerGame,Poss_PerGame,OppPoss_PerGame,OffEff_PerGame,DefEff_PerGame,OppOffEff_PerGame,OppDefEff_PerGame,Score_Diff_PerGame,FGM_Diff_PerGame,FGA_Diff_PerGame,FGM3_Diff_PerGame,FGA3_Diff_PerGame,FTM_Diff_PerGame,FTA_Diff_PerGame,OR_Diff_PerGame,DR_Diff_PerGame,Ast_Diff_PerGame,TO_Diff_PerGame,Stl_Diff_PerGame,Blk_Diff_PerGame,PF_Diff_PerGame,Poss_Diff_PerGame,OffEff_Diff_PerGame,DefEff_Diff_PerGame,Score_Diff_Pct_PerGame,FGM_Diff_Pct_PerGame,FGA_Diff_Pct_PerGame,FGM3_Diff_Pct_PerGame,FGA3_Diff_Pct_PerGame,FTM_Diff_Pct_PerGame,FTA_Diff_Pct_PerGame,OR_Diff_Pct_PerGame,DR_Diff_Pct_PerGame,Ast_Diff_Pct_PerGame,TO_Diff_Pct_PerGame,Stl_Diff_Pct_PerGame,Blk_Diff_Pct_PerGame,PF_Diff_Pct_PerGame,Poss_Diff_Pct_PerGame,OffEff_Diff_Pct_PerGame,DefEff_Diff_Pct_PerGame,Score_Diff_Opp_PerGame,FGM_Diff_Opp_PerGame,FGA_Diff_Opp_PerGame,FGM3_Diff_Opp_PerGame,FGA3_Diff_Opp_PerGame,FTM_Diff_Opp_PerGame,FTA_Diff_Opp_PerGame,OR_Diff_Opp_PerGame,DR_Diff_Opp_PerGame,Ast_Diff_Opp_PerGame,TO_Diff_Opp_PerGame,Stl_Diff_Opp_PerGame,Blk_Diff_Opp_PerGame,PF_Diff_Opp_PerGame,Poss_Diff_Opp_PerGame,OffEff_Diff_Opp_PerGame,DefEff_Diff_Opp_PerGame,Score_Diff_Pct_Opp_PerGame,FGM_Diff_Pct_Opp_PerGame,FGA_Diff_Pct_Opp_PerGame,FGM3_Diff_Pct_Opp_PerGame,FGA3_Diff_Pct_Opp_PerGame,FTM_Diff_Pct_Opp_PerGame,FTA_Diff_Pct_Opp_PerGame,OR_Diff_Pct_Opp_PerGame,DR_Diff_Pct_Opp_PerGame,Ast_Diff_Pct_Opp_PerGame,TO_Diff_Pct_Opp_PerGame,Stl_Diff_Pct_Opp_PerGame,Blk_Diff_Pct_Opp_PerGame,PF_Diff_Pct_Opp_PerGame,Poss_Diff_Pct_Opp_PerGame,OffEff_Diff_Pct_Opp_PerGame,DefEff_Diff_Pct_Opp_PerGame,KenPomRank_PerGame,OppKenPomRank_PerGame,KenPomDiff_PerGame,OppKenPomDiff_PerGame,Score_PerWin,OppScore_PerWin,NumOT_PerWin,FGM_PerWin,FGA_PerWin,FGM3_PerWin,FGA3_PerWin,FTM_PerWin,FTA_PerWin,OR_PerWin,DR_PerWin,Ast_PerWin,TO_PerWin,Stl_PerWin,Blk_PerWin,PF_PerWin,OppFGM_PerWin,OppFGA_PerWin,OppFGM3_PerWin,OppFGA3_PerWin,OppFTM_PerWin,OppFTA_PerWin,OppOR_PerWin,OppDR_PerWin,OppAst_PerWin,OppTO_PerWin,OppStl_PerWin,OppBlk_PerWin,OppPF_PerWin,Win_PerWin,FG_Percentage_PerWin,FG3_Percentage_PerWin,FT_Percentage_PerWin,Ast_TO_Ratio_PerWin,OppFG_Percentage_PerWin,OppFG3_Percentage_PerWin,OppFT_Percentage_PerWin,OppAst_TO_Ratio_PerWin,Poss_PerWin,OppPoss_PerWin,OffEff_PerWin,DefEff_PerWin,OppOffEff_PerWin,OppDefEff_PerWin,Score_Diff_PerWin,FGM_Diff_PerWin,FGA_Diff_PerWin,FGM3_Diff_PerWin,FGA3_Diff_PerWin,FTM_Diff_PerWin,FTA_Diff_PerWin,OR_Diff_PerWin,DR_Diff_PerWin,Ast_Diff_PerWin,TO_Diff_PerWin,Stl_Diff_PerWin,Blk_Diff_PerWin,PF_Diff_PerWin,Poss_Diff_PerWin,OffEff_Diff_PerWin,DefEff_Diff_PerWin,Score_Diff_Pct_PerWin,FGM_Diff_Pct_PerWin,FGA_Diff_Pct_PerWin,FGM3_Diff_Pct_PerWin,FGA3_Diff_Pct_PerWin,FTM_Diff_Pct_PerWin,FTA_Diff_Pct_PerWin,OR_Diff_Pct_PerWin,DR_Diff_Pct_PerWin,Ast_Diff_Pct_PerWin,TO_Diff_Pct_PerWin,Stl_Diff_Pct_PerWin,Blk_Diff_Pct_PerWin,PF_Diff_Pct_PerWin,Poss_Diff_Pct_PerWin,OffEff_Diff_Pct_PerWin,DefEff_Diff_Pct_PerWin,Score_Diff_Opp_PerWin,FGM_Diff_Opp_PerWin,FGA_Diff_Opp_PerWin,FGM3_Diff_Opp_PerWin,FGA3_Diff_Opp_PerWin,FTM_Diff_Opp_PerWin,FTA_Diff_Opp_PerWin,OR_Diff_Opp_PerWin,DR_Diff_Opp_PerWin,Ast_Diff_Opp_PerWin,TO_Diff_Opp_PerWin,Stl_Diff_Opp_PerWin,Blk_Diff_Opp_PerWin,PF_Diff_Opp_PerWin,Poss_Diff_Opp_PerWin,OffEff_Diff_Opp_PerWin,DefEff_Diff_Opp_PerWin,Score_Diff_Pct_Opp_PerWin,FGM_Diff_Pct_Opp_PerWin,FGA_Diff_Pc

***
### Tournament Matchups

- Using the `MNCAATourneyCompactResults.csv`, we will reshape the dataset into the same format as done previously, creating two (2) records per game with the `WTeamID` and `LTeamID` as the the main `TeamID`.
- The the regular season features will be merged on the `TeamID` and `OppTeamID` to get a robust set of features to train our model on.
- This will result in a binary classification problem for each game, with the target being the `'Win'` column (`win=1` | `loss = 0`).

***

In [565]:
tournament_results = pd.read_csv('./Data/MNCAATourneyCompactResults.csv', dtype={
    'WTeamID':str,
    'LTeamID': str}
    )

In [566]:
print(tournament_results.shape)
display(tournament_results.head())
print(tournament_results.info())

(2518, 8)


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2518 entries, 0 to 2517
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Season   2518 non-null   int64 
 1   DayNum   2518 non-null   int64 
 2   WTeamID  2518 non-null   object
 3   WScore   2518 non-null   int64 
 4   LTeamID  2518 non-null   object
 5   LScore   2518 non-null   int64 
 6   WLoc     2518 non-null   object
 7   NumOT    2518 non-null   int64 
dtypes: int64(5), object(3)
memory usage: 157.5+ KB
None


In [567]:
max_tourney_season = tournament_results['Season'].max()
min_tourney_season = tournament_results['Season'].min()

max_season = regular_season_stats['Season'].max()
min_season = regular_season_stats['Season'].min()

print(f"Regular season results for last {regular_season_stats['Season'].nunique()} seasons ({min_season} - {max_season}).")
print(f"Tournament results for last {tournament_results['Season'].nunique()} seasons ({min_tourney_season} - {max_tourney_season}). No tournament in 2020 due to COVID-19.")

Regular season results for last 23 seasons (2003 - 2025).
Tournament results for last 39 seasons (1985 - 2024). No tournament in 2020 due to COVID-19.


Given the regular season data only contains information from `2003` onwards, we'll match the tournament dataset for the same.

In [568]:
tournament_results = tournament_results[tournament_results['Season'] >= 2003].reset_index(drop=True).copy()

In [569]:
df_team_tourney_results = pd.concat(
    [
        tournament_results
        .assign(GameResult="W")
        .rename(columns={"WTeamID": "TeamID", 
                         "LTeamID": "OppTeamID", 
                         "WScore": "TeamScore", 
                         "LScore": "OppScore"}),
        
        tournament_results
        .assign(GameResult="L")
        .rename(columns={"LTeamID": "TeamID", 
                         "WTeamID": "OppTeamID", 
                         "LScore": "TeamScore", 
                         "WScore": "OppScore"}),
    ]
).reset_index(drop=True)

In [570]:
df_team_tourney_results

,Season,DayNum,TeamID,TeamScore,OppTeamID,OppScore,WLoc,NumOT,GameResult
0,2003,134,1421,92,1411,84,N,1,W
1,2003,136,1112,80,1436,51,N,0,W
2,2003,136,1113,84,1272,71,N,0,W
3,2003,136,1141,79,1166,73,N,0,W
4,2003,136,1143,76,1301,74,N,1,W
...,...,...,...,...,...,...,...,...,...
2759,2024,146,1181,64,1301,76,N,0,L
2760,2024,146,1397,66,1345,72,N,0,L
2761,2024,152,1104,72,1163,86,N,0,L
2762,2024,152,1301,50,1345,63,N,0,L


In [571]:
df_team_tourney_results["Win"] = (df_team_tourney_results["GameResult"] == "W").astype("int")

df_team_tourney_results['TeamName'] = df_team_tourney_results['TeamID'].map(teams)
df_team_tourney_results['OppTeamName'] = df_team_tourney_results['OppTeamID'].map(teams)

df_team_tourney_results['OppLoc'] = df_team_tourney_results['WLoc'].map({'H': 'A', 'A': 'H', 'N': 'N'})
df_team_tourney_results = df_team_tourney_results.rename(columns={"WLoc": "Loc"})

In [572]:
# Quickly reorder columns
reorder = ['Season', 'DayNum', 'TeamID', 'TeamName', 'OppTeamID', 'OppTeamName']
df_team_tourney_results = df_team_tourney_results[reorder + [col for col in df_team_tourney_results.columns if col not in reorder]]

df_team_tourney_results

,Season,DayNum,TeamID,TeamName,OppTeamID,OppTeamName,TeamScore,OppScore,Loc,NumOT,GameResult,Win,OppLoc
0,2003,134,1421,UNC Asheville,1411,TX Southern,92,84,N,1,W,1,N
1,2003,136,1112,Arizona,1436,Vermont,80,51,N,0,W,1,N
2,2003,136,1113,Arizona St,1272,Memphis,84,71,N,0,W,1,N
3,2003,136,1141,C Michigan,1166,Creighton,79,73,N,0,W,1,N
4,2003,136,1143,California,1301,NC State,76,74,N,1,W,1,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2759,2024,146,1181,Duke,1301,NC State,64,76,N,0,L,0,N
2760,2024,146,1397,Tennessee,1345,Purdue,66,72,N,0,L,0,N
2761,2024,152,1104,Alabama,1163,Connecticut,72,86,N,0,L,0,N
2762,2024,152,1301,NC State,1345,Purdue,50,63,N,0,L,0,N


In [573]:
df_team_tourney_results['Loc'].value_counts()

Loc
N    2764
Name: count, dtype: int64

While NCAA Tournament games are officially played at neutral sites (`Loc == 'N'`), there have been notable instances over the last 25 years where one team likely benefitted from a **'home-court'** advantage, including back-to-back championship games (2009 - 2010) where location strongly favored one team.

| Year  | Round                      | Matchup                        | Venue & Location                  | Details                                                                                                      |
|-------|----------------------------|--------------------------------|-----------------------------------|--------------------------------------------------------------------------------------------------------------|
| 2005  | Elite Eight                | Illinois vs. Arizona           | Allstate Arena (Chicago, IL)    | Held two hours from Illinois’ campus, generating a predominantly Illini crowd that played a critical role in their 15-point comeback win. |
| 2009  | Final Four & Championship  | Michigan State vs. UConn & UNC | Ford Field (Detroit, MI)         | Michigan State played both the Final Four and National Championship in its own backyard. Despite the home crowd advantage, MSU fell to UNC in the title game. |
| 2010  | Championship               | Butler vs. Duke                | Lucas Oil Stadium (Indianapolis, IN) | Held in Butler’s home city, Butler pushed #1 seed Duke to the final possession, narrowly missing a championship-winning shot at the buzzer. |
| 2014  | Round of 32                | Wisconsin vs. Oregon           | BMO Harris Center (Milwaukee, WI) | Game hosted in Wisconsin, resulting in a home-like environment with overwhelming Badger fan support. |
| 2015  | First Four                 | Dayton vs. Boise State         | UD Arena (Dayton, OH)            | Game played on Dayton’s home court, creating a true home-court advantage. First NCAA Tournament game on a school’s home floor since 1987. |
| 2017  | Second Round               | South Carolina vs. Duke        | Bon Secours Arena (Greenville, SC) | Game held in South Carolina, leading to an overwhelmingly SC-dominated crowd. |
| 2017  | Sweet 16 & Elite Eight     | Kansas vs. Purdue & Oregon     | Sprint Center (Kansas City, MO) | Only 40 miles from Kansas’ campus, providing a significant competitive advantage in the S16 and E8. |




Because all games are tagged as 'neutral' in the tournament dataset, we can drop the location columns, as they will not be informative to the model.

In [574]:
df_team_tourney_results = df_team_tourney_results.drop(columns=['Loc', 'OppLoc'])

In [575]:
df_team_tourney_results

,Season,DayNum,TeamID,TeamName,OppTeamID,OppTeamName,TeamScore,OppScore,NumOT,GameResult,Win
0,2003,134,1421,UNC Asheville,1411,TX Southern,92,84,1,W,1
1,2003,136,1112,Arizona,1436,Vermont,80,51,0,W,1
2,2003,136,1113,Arizona St,1272,Memphis,84,71,0,W,1
3,2003,136,1141,C Michigan,1166,Creighton,79,73,0,W,1
4,2003,136,1143,California,1301,NC State,76,74,1,W,1
...,...,...,...,...,...,...,...,...,...,...,...
2759,2024,146,1181,Duke,1301,NC State,64,76,0,L,0
2760,2024,146,1397,Tennessee,1345,Purdue,66,72,0,L,0
2761,2024,152,1104,Alabama,1163,Connecticut,72,86,0,L,0
2762,2024,152,1301,NC State,1345,Purdue,50,63,0,L,0


Now that we have the skelton for the historical tournament matchups, we'll merge the regular season stats for each `TeamID` and `OppTeamID`.

In [576]:
tournament_df = df_team_tourney_results.merge(
    regular_season_stats.drop(columns=["TeamName"]), 
    on=["Season", "TeamID"], 
    how="left"
).merge(
    regular_season_stats.drop(columns=["TeamName"]), 
    left_on=["Season", "OppTeamID"], 
    right_on=["Season", "TeamID"], 
    suffixes=("", "_Opp"),
    how="left"
).sort_values(['Season', 'DayNum', 'TeamID']).reset_index(drop=True)

In [577]:
print(tournament_df.shape)
display(tournament_df.tail())

(2764, 726)


,Season,DayNum,TeamID,TeamName,OppTeamID,OppTeamName,TeamScore,OppScore,NumOT,GameResult,Win,ConfAbbrev,Description,TotalGames,TotalWins,TotalLosses,WinningPercentage,Score_PerGame,OppScore_PerGame,NumOT_PerGame,FGM_PerGame,FGA_PerGame,FGM3_PerGame,FGA3_PerGame,FTM_PerGame,FTA_PerGame,OR_PerGame,DR_PerGame,Ast_PerGame,TO_PerGame,Stl_PerGame,Blk_PerGame,PF_PerGame,OppFGM_PerGame,OppFGA_PerGame,OppFGM3_PerGame,OppFGA3_PerGame,OppFTM_PerGame,OppFTA_PerGame,OppOR_PerGame,OppDR_PerGame,OppAst_PerGame,OppTO_PerGame,OppStl_PerGame,OppBlk_PerGame,OppPF_PerGame,Win_PerGame,FG_Percentage_PerGame,FG3_Percentage_PerGame,FT_Percentage_PerGame,Ast_TO_Ratio_PerGame,OppFG_Percentage_PerGame,OppFG3_Percentage_PerGame,OppFT_Percentage_PerGame,OppAst_TO_Ratio_PerGame,Poss_PerGame,OppPoss_PerGame,OffEff_PerGame,DefEff_PerGame,OppOffEff_PerGame,OppDefEff_PerGame,Score_Diff_PerGame,FGM_Diff_PerGame,FGA_Diff_PerGame,FGM3_Diff_PerGame,FGA3_Diff_PerGame,FTM_Diff_PerGame,FTA_Diff_PerGame,OR_Diff_PerGame,DR_Diff_PerGame,Ast_Diff_PerGame,TO_Diff_PerGame,Stl_Diff_PerGame,Blk_Diff_PerGame,PF_Diff_PerGame,Poss_Diff_PerGame,OffEff_Diff_PerGame,DefEff_Diff_PerGame,Score_Diff_Pct_PerGame,FGM_Diff_Pct_PerGame,FGA_Diff_Pct_PerGame,FGM3_Diff_Pct_PerGame,FGA3_Diff_Pct_PerGame,FTM_Diff_Pct_PerGame,FTA_Diff_Pct_PerGame,OR_Diff_Pct_PerGame,DR_Diff_Pct_PerGame,Ast_Diff_Pct_PerGame,TO_Diff_Pct_PerGame,Stl_Diff_Pct_PerGame,Blk_Diff_Pct_PerGame,PF_Diff_Pct_PerGame,Poss_Diff_Pct_PerGame,OffEff_Diff_Pct_PerGame,DefEff_Diff_Pct_PerGame,Score_Diff_Opp_PerGame,FGM_Diff_Opp_PerGame,FGA_Diff_Opp_PerGame,FGM3_Diff_Opp_PerGame,FGA3_Diff_Opp_PerGame,FTM_Diff_Opp_PerGame,FTA_Diff_Opp_PerGame,OR_Diff_Opp_PerGame,DR_Diff_Opp_PerGame,Ast_Diff_Opp_PerGame,TO_Diff_Opp_PerGame,Stl_Diff_Opp_PerGame,Blk_Diff_Opp_PerGame,PF_Diff_Opp_PerGame,Poss_Diff_Opp_PerGame,OffEff_Diff_Opp_PerGame,DefEff_Diff_Opp_PerGame,Score_Diff_Pct_Opp_PerGame,FGM_Diff_Pct_Opp_PerGame,FGA_Diff_Pct_Opp_PerGame,FGM3_Diff_Pct_Opp_PerGame,FGA3_Diff_Pct_Opp_PerGame,FTM_Diff_Pct_Opp_PerGame,FTA_Diff_Pct_Opp_PerGame,OR_Diff_Pct_Opp_PerGame,DR_Diff_Pct_Opp_PerGame,Ast_Diff_Pct_Opp_PerGame,TO_Diff_Pct_Opp_PerGame,Stl_Diff_Pct_Opp_PerGame,Blk_Diff_Pct_Opp_PerGame,PF_Diff_Pct_Opp_PerGame,Poss_Diff_Pct_Opp_PerGame,OffEff_Diff_Pct_Opp_PerGame,DefEff_Diff_Pct_Opp_PerGame,KenPomRank_PerGame,OppKenPomRank_PerGame,KenPomDiff_PerGame,OppKenPomDiff_PerGame,Score_PerWin,OppScore_PerWin,NumOT_PerWin,FGM_PerWin,FGA_PerWin,FGM3_PerWin,FGA3_PerWin,FTM_PerWin,FTA_PerWin,OR_PerWin,DR_PerWin,Ast_PerWin,TO_PerWin,Stl_PerWin,Blk_PerWin,PF_PerWin,OppFGM_PerWin,OppFGA_PerWin,OppFGM3_PerWin,OppFGA3_PerWin,OppFTM_PerWin,OppFTA_PerWin,OppOR_PerWin,OppDR_PerWin,OppAst_PerWin,OppTO_PerWin,OppStl_PerWin,OppBlk_PerWin,OppPF_PerWin,Win_PerWin,FG_Percentage_PerWin,FG3_Percentage_PerWin,FT_Percentage_PerWin,Ast_TO_Ratio_PerWin,OppFG_Percentage_PerWin,OppFG3_Percentage_PerWin,OppFT_Percentage_PerWin,OppAst_TO_Ratio_PerWin,Poss_PerWin,OppPoss_PerWin,OffEff_PerWin,DefEff_PerWin,OppOffEff_PerWin,OppDefEff_PerWin,Score_Diff_PerWin,FGM_Diff_PerWin,FGA_Diff_PerWin,FGM3_Diff_PerWin,FGA3_Diff_PerWin,FTM_Diff_PerWin,FTA_Diff_PerWin,OR_Diff_PerWin,DR_Diff_PerWin,Ast_Diff_PerWin,TO_Diff_PerWin,Stl_Diff_PerWin,Blk_Diff_PerWin,PF_Diff_PerWin,Poss_Diff_PerWin,OffEff_Diff_PerWin,DefEff_Diff_PerWin,Score_Diff_Pct_PerWin,FGM_Diff_Pct_PerWin,FGA_Diff_Pct_PerWin,FGM3_Diff_Pct_PerWin,FGA3_Diff_Pct_PerWin,FTM_Diff_Pct_PerWin,FTA_Diff_Pct_PerWin,OR_Diff_Pct_PerWin,DR_Diff_Pct_PerWin,Ast_Diff_Pct_PerWin,TO_Diff_Pct_PerWin,Stl_Diff_Pct_PerWin,Blk_Diff_Pct_PerWin,PF_Diff_Pct_PerWin,Poss_Diff_Pct_PerWin,OffEff_Diff_Pct_PerWin,DefEff_Diff_Pct_PerWin,Score_Diff_Opp_PerWin,FGM_Diff_Opp_PerWin,FGA_Diff_Opp_PerWin,FGM3_Diff_Opp_PerWin,FGA3_Diff_Opp_PerWin,FTM_Diff_Opp_PerWin,FTA_Diff_Opp_PerWin,OR_Diff_Opp_PerWin,DR_Diff_Opp_PerWin,Ast_Diff_Opp_PerWin,TO_Diff_Opp_PerWin,Stl_Diff_Opp_PerWin,Blk_Diff_Opp_PerWin,PF_Diff_Opp_PerWin,Poss_Diff_Opp_PerWin,OffEff_Diff_Opp_PerWin,DefEff_Diff_Opp

In [578]:
print(f"Number of rows with any missing values: {tournament_df[tournament_df.isna().any(axis=1)].shape[0]}")
print(f"Number of rows with missing 'SeedNum' (whether a team made the tournament): {tournament_df[tournament_df['SeedNum'].isna()].shape[0]}")
print(f"Total number of missing values in rows where 'Seed' is not missing: {tournament_df[~(tournament_df['Seed'].isna())].isna().sum().sum()}")

Number of rows with any missing values: 0
Number of rows with missing 'SeedNum' (whether a team made the tournament): 0
Total number of missing values in rows where 'Seed' is not missing: 0


Based on the above, the missing values across the dataset are driven by teams that did not make the tournament in a given season.

***

## Feature Engineering: Matchups

***

In [579]:
# Drop TeamID_Opp same as OppTeamID
tournament_df = tournament_df.drop(columns=['TeamID_Opp'])

In [580]:
# Extract object cols and _Opp columns
object_cols = tournament_df.select_dtypes(include=['object']).columns.tolist()
opp_columns = [col for col in tournament_df.columns if col.endswith('_Opp') and col not in object_cols]

print(object_cols)
print(opp_columns)

['TeamID', 'TeamName', 'OppTeamID', 'OppTeamName', 'GameResult', 'ConfAbbrev', 'Description', 'Seed', 'Region', 'ConfAbbrev_Opp', 'Description_Opp', 'Seed_Opp', 'Region_Opp']
['TotalGames_Opp', 'TotalWins_Opp', 'TotalLosses_Opp', 'WinningPercentage_Opp', 'Score_PerGame_Opp', 'OppScore_PerGame_Opp', 'NumOT_PerGame_Opp', 'FGM_PerGame_Opp', 'FGA_PerGame_Opp', 'FGM3_PerGame_Opp', 'FGA3_PerGame_Opp', 'FTM_PerGame_Opp', 'FTA_PerGame_Opp', 'OR_PerGame_Opp', 'DR_PerGame_Opp', 'Ast_PerGame_Opp', 'TO_PerGame_Opp', 'Stl_PerGame_Opp', 'Blk_PerGame_Opp', 'PF_PerGame_Opp', 'OppFGM_PerGame_Opp', 'OppFGA_PerGame_Opp', 'OppFGM3_PerGame_Opp', 'OppFGA3_PerGame_Opp', 'OppFTM_PerGame_Opp', 'OppFTA_PerGame_Opp', 'OppOR_PerGame_Opp', 'OppDR_PerGame_Opp', 'OppAst_PerGame_Opp', 'OppTO_PerGame_Opp', 'OppStl_PerGame_Opp', 'OppBlk_PerGame_Opp', 'OppPF_PerGame_Opp', 'Win_PerGame_Opp', 'FG_Percentage_PerGame_Opp', 'FG3_Percentage_PerGame_Opp', 'FT_Percentage_PerGame_Opp', 'Ast_TO_Ratio_PerGame_Opp', 'OppFG_Percenta

In [581]:
# Loop through opponent columns and calculate deltas
delta_dict = {}

for col in opp_columns:
    base_col = col.replace("_Opp", "")  # Remove '_Opp' to get the original column name
    if base_col in tournament_df.columns:
        delta_dict[f"{base_col}_delta"] = tournament_df[base_col] - tournament_df[col]

In [582]:
# Convert dictionary to DataFrame and concatenate
delta_df = pd.DataFrame(delta_dict, index=tournament_df.index)
tournament_df = pd.concat([tournament_df, delta_df], axis=1)

In [583]:
delta_df.shape

(2764, 251)

In [584]:
print(tournament_df.shape)
display(tournament_df.tail())

(2764, 976)


,Season,DayNum,TeamID,TeamName,OppTeamID,OppTeamName,TeamScore,OppScore,NumOT,GameResult,Win,ConfAbbrev,Description,TotalGames,TotalWins,TotalLosses,WinningPercentage,Score_PerGame,OppScore_PerGame,NumOT_PerGame,FGM_PerGame,FGA_PerGame,FGM3_PerGame,FGA3_PerGame,FTM_PerGame,FTA_PerGame,OR_PerGame,DR_PerGame,Ast_PerGame,TO_PerGame,Stl_PerGame,Blk_PerGame,PF_PerGame,OppFGM_PerGame,OppFGA_PerGame,OppFGM3_PerGame,OppFGA3_PerGame,OppFTM_PerGame,OppFTA_PerGame,OppOR_PerGame,OppDR_PerGame,OppAst_PerGame,OppTO_PerGame,OppStl_PerGame,OppBlk_PerGame,OppPF_PerGame,Win_PerGame,FG_Percentage_PerGame,FG3_Percentage_PerGame,FT_Percentage_PerGame,Ast_TO_Ratio_PerGame,OppFG_Percentage_PerGame,OppFG3_Percentage_PerGame,OppFT_Percentage_PerGame,OppAst_TO_Ratio_PerGame,Poss_PerGame,OppPoss_PerGame,OffEff_PerGame,DefEff_PerGame,OppOffEff_PerGame,OppDefEff_PerGame,Score_Diff_PerGame,FGM_Diff_PerGame,FGA_Diff_PerGame,FGM3_Diff_PerGame,FGA3_Diff_PerGame,FTM_Diff_PerGame,FTA_Diff_PerGame,OR_Diff_PerGame,DR_Diff_PerGame,Ast_Diff_PerGame,TO_Diff_PerGame,Stl_Diff_PerGame,Blk_Diff_PerGame,PF_Diff_PerGame,Poss_Diff_PerGame,OffEff_Diff_PerGame,DefEff_Diff_PerGame,Score_Diff_Pct_PerGame,FGM_Diff_Pct_PerGame,FGA_Diff_Pct_PerGame,FGM3_Diff_Pct_PerGame,FGA3_Diff_Pct_PerGame,FTM_Diff_Pct_PerGame,FTA_Diff_Pct_PerGame,OR_Diff_Pct_PerGame,DR_Diff_Pct_PerGame,Ast_Diff_Pct_PerGame,TO_Diff_Pct_PerGame,Stl_Diff_Pct_PerGame,Blk_Diff_Pct_PerGame,PF_Diff_Pct_PerGame,Poss_Diff_Pct_PerGame,OffEff_Diff_Pct_PerGame,DefEff_Diff_Pct_PerGame,Score_Diff_Opp_PerGame,FGM_Diff_Opp_PerGame,FGA_Diff_Opp_PerGame,FGM3_Diff_Opp_PerGame,FGA3_Diff_Opp_PerGame,FTM_Diff_Opp_PerGame,FTA_Diff_Opp_PerGame,OR_Diff_Opp_PerGame,DR_Diff_Opp_PerGame,Ast_Diff_Opp_PerGame,TO_Diff_Opp_PerGame,Stl_Diff_Opp_PerGame,Blk_Diff_Opp_PerGame,PF_Diff_Opp_PerGame,Poss_Diff_Opp_PerGame,OffEff_Diff_Opp_PerGame,DefEff_Diff_Opp_PerGame,Score_Diff_Pct_Opp_PerGame,FGM_Diff_Pct_Opp_PerGame,FGA_Diff_Pct_Opp_PerGame,FGM3_Diff_Pct_Opp_PerGame,FGA3_Diff_Pct_Opp_PerGame,FTM_Diff_Pct_Opp_PerGame,FTA_Diff_Pct_Opp_PerGame,OR_Diff_Pct_Opp_PerGame,DR_Diff_Pct_Opp_PerGame,Ast_Diff_Pct_Opp_PerGame,TO_Diff_Pct_Opp_PerGame,Stl_Diff_Pct_Opp_PerGame,Blk_Diff_Pct_Opp_PerGame,PF_Diff_Pct_Opp_PerGame,Poss_Diff_Pct_Opp_PerGame,OffEff_Diff_Pct_Opp_PerGame,DefEff_Diff_Pct_Opp_PerGame,KenPomRank_PerGame,OppKenPomRank_PerGame,KenPomDiff_PerGame,OppKenPomDiff_PerGame,Score_PerWin,OppScore_PerWin,NumOT_PerWin,FGM_PerWin,FGA_PerWin,FGM3_PerWin,FGA3_PerWin,FTM_PerWin,FTA_PerWin,OR_PerWin,DR_PerWin,Ast_PerWin,TO_PerWin,Stl_PerWin,Blk_PerWin,PF_PerWin,OppFGM_PerWin,OppFGA_PerWin,OppFGM3_PerWin,OppFGA3_PerWin,OppFTM_PerWin,OppFTA_PerWin,OppOR_PerWin,OppDR_PerWin,OppAst_PerWin,OppTO_PerWin,OppStl_PerWin,OppBlk_PerWin,OppPF_PerWin,Win_PerWin,FG_Percentage_PerWin,FG3_Percentage_PerWin,FT_Percentage_PerWin,Ast_TO_Ratio_PerWin,OppFG_Percentage_PerWin,OppFG3_Percentage_PerWin,OppFT_Percentage_PerWin,OppAst_TO_Ratio_PerWin,Poss_PerWin,OppPoss_PerWin,OffEff_PerWin,DefEff_PerWin,OppOffEff_PerWin,OppDefEff_PerWin,Score_Diff_PerWin,FGM_Diff_PerWin,FGA_Diff_PerWin,FGM3_Diff_PerWin,FGA3_Diff_PerWin,FTM_Diff_PerWin,FTA_Diff_PerWin,OR_Diff_PerWin,DR_Diff_PerWin,Ast_Diff_PerWin,TO_Diff_PerWin,Stl_Diff_PerWin,Blk_Diff_PerWin,PF_Diff_PerWin,Poss_Diff_PerWin,OffEff_Diff_PerWin,DefEff_Diff_PerWin,Score_Diff_Pct_PerWin,FGM_Diff_Pct_PerWin,FGA_Diff_Pct_PerWin,FGM3_Diff_Pct_PerWin,FGA3_Diff_Pct_PerWin,FTM_Diff_Pct_PerWin,FTA_Diff_Pct_PerWin,OR_Diff_Pct_PerWin,DR_Diff_Pct_PerWin,Ast_Diff_Pct_PerWin,TO_Diff_Pct_PerWin,Stl_Diff_Pct_PerWin,Blk_Diff_Pct_PerWin,PF_Diff_Pct_PerWin,Poss_Diff_Pct_PerWin,OffEff_Diff_Pct_PerWin,DefEff_Diff_Pct_PerWin,Score_Diff_Opp_PerWin,FGM_Diff_Opp_PerWin,FGA_Diff_Opp_PerWin,FGM3_Diff_Opp_PerWin,FGA3_Diff_Opp_PerWin,FTM_Diff_Opp_PerWin,FTA_Diff_Opp_PerWin,OR_Diff_Opp_PerWin,DR_Diff_Opp_PerWin,Ast_Diff_Opp_PerWin,TO_Diff_Opp_PerWin,Stl_Diff_Opp_PerWin,Blk_Diff_Opp_PerWin,PF_Diff_Opp_PerWin,Poss_Diff_Opp_PerWin,OffEff_Diff_Opp_PerWin,DefEff_Diff_Opp

Let's also include the final `KenPomRank` as of the last day of the regular season. This metric serves as a strong proxy for overall team quality and provides valuable context heading into the tournament.

In [585]:
ordinals_df = pd.read_csv('./Data/MMasseyOrdinals.csv', dtype={'TeamID': str})

# Extract KenPom metrics
kenpom = ordinals_df[ordinals_df['SystemName'] == 'POM'].rename(columns={
    'RankingDayNum': 'DayNum',
    'OrdinalRank': 'KenPomRank'}).drop(columns='SystemName')

# kenpom.groupby(['Season'])['DayNum'].max() = 133

# Df of final KenPomRanks
kenpom = kenpom[kenpom['DayNum'] == 133]

In [586]:
# Merge team KenPom rank
tournament_df = tournament_df.merge(
    kenpom.drop(columns='DayNum'),
    on=['Season', 'TeamID'],
    how='left'
)

tournament_df = tournament_df.merge(
    kenpom.rename(columns={
        'TeamID': 'OppTeamID',
        'KenPomRank': 'OppKenPomRank'
    }).drop(columns='DayNum'),
    on=['Season', 'OppTeamID'],
    how='left'
)

In [587]:
reorder = [
    'Season', 'DayNum', 'TeamID', 'OppTeamID',
    'TeamScore', 'OppScore','GameResult', 'Win',
    'TeamName', 'OppTeamName', 'KenPomRank', 'OppKenPomRank',
    'ConfAbbrev', 'Description', 'ConfAbbrev_Opp', 'Description_Opp'
    ]

tournament_df = tournament_df[reorder + [col for col in tournament_df.columns if col not in reorder]]
tournament_df.tail()

,Season,DayNum,TeamID,OppTeamID,TeamScore,OppScore,GameResult,Win,TeamName,OppTeamName,KenPomRank,OppKenPomRank,ConfAbbrev,Description,ConfAbbrev_Opp,Description_Opp,NumOT,TotalGames,TotalWins,TotalLosses,WinningPercentage,Score_PerGame,OppScore_PerGame,NumOT_PerGame,FGM_PerGame,FGA_PerGame,FGM3_PerGame,FGA3_PerGame,FTM_PerGame,FTA_PerGame,OR_PerGame,DR_PerGame,Ast_PerGame,TO_PerGame,Stl_PerGame,Blk_PerGame,PF_PerGame,OppFGM_PerGame,OppFGA_PerGame,OppFGM3_PerGame,OppFGA3_PerGame,OppFTM_PerGame,OppFTA_PerGame,OppOR_PerGame,OppDR_PerGame,OppAst_PerGame,OppTO_PerGame,OppStl_PerGame,OppBlk_PerGame,OppPF_PerGame,Win_PerGame,FG_Percentage_PerGame,FG3_Percentage_PerGame,FT_Percentage_PerGame,Ast_TO_Ratio_PerGame,OppFG_Percentage_PerGame,OppFG3_Percentage_PerGame,OppFT_Percentage_PerGame,OppAst_TO_Ratio_PerGame,Poss_PerGame,OppPoss_PerGame,OffEff_PerGame,DefEff_PerGame,OppOffEff_PerGame,OppDefEff_PerGame,Score_Diff_PerGame,FGM_Diff_PerGame,FGA_Diff_PerGame,FGM3_Diff_PerGame,FGA3_Diff_PerGame,FTM_Diff_PerGame,FTA_Diff_PerGame,OR_Diff_PerGame,DR_Diff_PerGame,Ast_Diff_PerGame,TO_Diff_PerGame,Stl_Diff_PerGame,Blk_Diff_PerGame,PF_Diff_PerGame,Poss_Diff_PerGame,OffEff_Diff_PerGame,DefEff_Diff_PerGame,Score_Diff_Pct_PerGame,FGM_Diff_Pct_PerGame,FGA_Diff_Pct_PerGame,FGM3_Diff_Pct_PerGame,FGA3_Diff_Pct_PerGame,FTM_Diff_Pct_PerGame,FTA_Diff_Pct_PerGame,OR_Diff_Pct_PerGame,DR_Diff_Pct_PerGame,Ast_Diff_Pct_PerGame,TO_Diff_Pct_PerGame,Stl_Diff_Pct_PerGame,Blk_Diff_Pct_PerGame,PF_Diff_Pct_PerGame,Poss_Diff_Pct_PerGame,OffEff_Diff_Pct_PerGame,DefEff_Diff_Pct_PerGame,Score_Diff_Opp_PerGame,FGM_Diff_Opp_PerGame,FGA_Diff_Opp_PerGame,FGM3_Diff_Opp_PerGame,FGA3_Diff_Opp_PerGame,FTM_Diff_Opp_PerGame,FTA_Diff_Opp_PerGame,OR_Diff_Opp_PerGame,DR_Diff_Opp_PerGame,Ast_Diff_Opp_PerGame,TO_Diff_Opp_PerGame,Stl_Diff_Opp_PerGame,Blk_Diff_Opp_PerGame,PF_Diff_Opp_PerGame,Poss_Diff_Opp_PerGame,OffEff_Diff_Opp_PerGame,DefEff_Diff_Opp_PerGame,Score_Diff_Pct_Opp_PerGame,FGM_Diff_Pct_Opp_PerGame,FGA_Diff_Pct_Opp_PerGame,FGM3_Diff_Pct_Opp_PerGame,FGA3_Diff_Pct_Opp_PerGame,FTM_Diff_Pct_Opp_PerGame,FTA_Diff_Pct_Opp_PerGame,OR_Diff_Pct_Opp_PerGame,DR_Diff_Pct_Opp_PerGame,Ast_Diff_Pct_Opp_PerGame,TO_Diff_Pct_Opp_PerGame,Stl_Diff_Pct_Opp_PerGame,Blk_Diff_Pct_Opp_PerGame,PF_Diff_Pct_Opp_PerGame,Poss_Diff_Pct_Opp_PerGame,OffEff_Diff_Pct_Opp_PerGame,DefEff_Diff_Pct_Opp_PerGame,KenPomRank_PerGame,OppKenPomRank_PerGame,KenPomDiff_PerGame,OppKenPomDiff_PerGame,Score_PerWin,OppScore_PerWin,NumOT_PerWin,FGM_PerWin,FGA_PerWin,FGM3_PerWin,FGA3_PerWin,FTM_PerWin,FTA_PerWin,OR_PerWin,DR_PerWin,Ast_PerWin,TO_PerWin,Stl_PerWin,Blk_PerWin,PF_PerWin,OppFGM_PerWin,OppFGA_PerWin,OppFGM3_PerWin,OppFGA3_PerWin,OppFTM_PerWin,OppFTA_PerWin,OppOR_PerWin,OppDR_PerWin,OppAst_PerWin,OppTO_PerWin,OppStl_PerWin,OppBlk_PerWin,OppPF_PerWin,Win_PerWin,FG_Percentage_PerWin,FG3_Percentage_PerWin,FT_Percentage_PerWin,Ast_TO_Ratio_PerWin,OppFG_Percentage_PerWin,OppFG3_Percentage_PerWin,OppFT_Percentage_PerWin,OppAst_TO_Ratio_PerWin,Poss_PerWin,OppPoss_PerWin,OffEff_PerWin,DefEff_PerWin,OppOffEff_PerWin,OppDefEff_PerWin,Score_Diff_PerWin,FGM_Diff_PerWin,FGA_Diff_PerWin,FGM3_Diff_PerWin,FGA3_Diff_PerWin,FTM_Diff_PerWin,FTA_Diff_PerWin,OR_Diff_PerWin,DR_Diff_PerWin,Ast_Diff_PerWin,TO_Diff_PerWin,Stl_Diff_PerWin,Blk_Diff_PerWin,PF_Diff_PerWin,Poss_Diff_PerWin,OffEff_Diff_PerWin,DefEff_Diff_PerWin,Score_Diff_Pct_PerWin,FGM_Diff_Pct_PerWin,FGA_Diff_Pct_PerWin,FGM3_Diff_Pct_PerWin,FGA3_Diff_Pct_PerWin,FTM_Diff_Pct_PerWin,FTA_Diff_Pct_PerWin,OR_Diff_Pct_PerWin,DR_Diff_Pct_PerWin,Ast_Diff_Pct_PerWin,TO_Diff_Pct_PerWin,Stl_Diff_Pct_PerWin,Blk_Diff_Pct_PerWin,PF_Diff_Pct_PerWin,Poss_Diff_Pct_PerWin,OffEff_Diff_Pct_PerWin,DefEff_Diff_Pct_PerWin,Score_Diff_Opp_PerWin,FGM_Diff_Opp_PerWin,FGA_Diff_Opp_PerWin,FGM3_Diff_Opp_PerWin,FGA3_Diff_Opp_PerWin,FTM_Diff_Opp_PerWin,FTA_Diff_Opp_PerWin,OR_Diff_Opp_PerWin,DR_Diff_Opp_PerWin,Ast_Diff_Opp_PerWin,TO_Diff_Opp_PerWin,Stl_Diff_Opp_PerWin,Blk_Diff_Opp_PerWin,PF_Diff_Opp_PerWin,Pos

In [588]:
# Save data to csv
tournament_df.to_csv('./Data/Live/TournamentMatchupData.csv', index=False)

In [589]:
regular_season_stats

,Season,TeamID,TeamName,ConfAbbrev,Description,TotalGames,TotalWins,TotalLosses,WinningPercentage,Score_PerGame,OppScore_PerGame,NumOT_PerGame,FGM_PerGame,FGA_PerGame,FGM3_PerGame,FGA3_PerGame,FTM_PerGame,FTA_PerGame,OR_PerGame,DR_PerGame,Ast_PerGame,TO_PerGame,Stl_PerGame,Blk_PerGame,PF_PerGame,OppFGM_PerGame,OppFGA_PerGame,OppFGM3_PerGame,OppFGA3_PerGame,OppFTM_PerGame,OppFTA_PerGame,OppOR_PerGame,OppDR_PerGame,OppAst_PerGame,OppTO_PerGame,OppStl_PerGame,OppBlk_PerGame,OppPF_PerGame,Win_PerGame,FG_Percentage_PerGame,FG3_Percentage_PerGame,FT_Percentage_PerGame,Ast_TO_Ratio_PerGame,OppFG_Percentage_PerGame,OppFG3_Percentage_PerGame,OppFT_Percentage_PerGame,OppAst_TO_Ratio_PerGame,Poss_PerGame,OppPoss_PerGame,OffEff_PerGame,DefEff_PerGame,OppOffEff_PerGame,OppDefEff_PerGame,Score_Diff_PerGame,FGM_Diff_PerGame,FGA_Diff_PerGame,FGM3_Diff_PerGame,FGA3_Diff_PerGame,FTM_Diff_PerGame,FTA_Diff_PerGame,OR_Diff_PerGame,DR_Diff_PerGame,Ast_Diff_PerGame,TO_Diff_PerGame,Stl_Diff_PerGame,Blk_Diff_PerGame,PF_Diff_PerGame,Poss_Diff_PerGame,OffEff_Diff_PerGame,DefEff_Diff_PerGame,Score_Diff_Pct_PerGame,FGM_Diff_Pct_PerGame,FGA_Diff_Pct_PerGame,FGM3_Diff_Pct_PerGame,FGA3_Diff_Pct_PerGame,FTM_Diff_Pct_PerGame,FTA_Diff_Pct_PerGame,OR_Diff_Pct_PerGame,DR_Diff_Pct_PerGame,Ast_Diff_Pct_PerGame,TO_Diff_Pct_PerGame,Stl_Diff_Pct_PerGame,Blk_Diff_Pct_PerGame,PF_Diff_Pct_PerGame,Poss_Diff_Pct_PerGame,OffEff_Diff_Pct_PerGame,DefEff_Diff_Pct_PerGame,Score_Diff_Opp_PerGame,FGM_Diff_Opp_PerGame,FGA_Diff_Opp_PerGame,FGM3_Diff_Opp_PerGame,FGA3_Diff_Opp_PerGame,FTM_Diff_Opp_PerGame,FTA_Diff_Opp_PerGame,OR_Diff_Opp_PerGame,DR_Diff_Opp_PerGame,Ast_Diff_Opp_PerGame,TO_Diff_Opp_PerGame,Stl_Diff_Opp_PerGame,Blk_Diff_Opp_PerGame,PF_Diff_Opp_PerGame,Poss_Diff_Opp_PerGame,OffEff_Diff_Opp_PerGame,DefEff_Diff_Opp_PerGame,Score_Diff_Pct_Opp_PerGame,FGM_Diff_Pct_Opp_PerGame,FGA_Diff_Pct_Opp_PerGame,FGM3_Diff_Pct_Opp_PerGame,FGA3_Diff_Pct_Opp_PerGame,FTM_Diff_Pct_Opp_PerGame,FTA_Diff_Pct_Opp_PerGame,OR_Diff_Pct_Opp_PerGame,DR_Diff_Pct_Opp_PerGame,Ast_Diff_Pct_Opp_PerGame,TO_Diff_Pct_Opp_PerGame,Stl_Diff_Pct_Opp_PerGame,Blk_Diff_Pct_Opp_PerGame,PF_Diff_Pct_Opp_PerGame,Poss_Diff_Pct_Opp_PerGame,OffEff_Diff_Pct_Opp_PerGame,DefEff_Diff_Pct_Opp_PerGame,KenPomRank_PerGame,OppKenPomRank_PerGame,KenPomDiff_PerGame,OppKenPomDiff_PerGame,Score_PerWin,OppScore_PerWin,NumOT_PerWin,FGM_PerWin,FGA_PerWin,FGM3_PerWin,FGA3_PerWin,FTM_PerWin,FTA_PerWin,OR_PerWin,DR_PerWin,Ast_PerWin,TO_PerWin,Stl_PerWin,Blk_PerWin,PF_PerWin,OppFGM_PerWin,OppFGA_PerWin,OppFGM3_PerWin,OppFGA3_PerWin,OppFTM_PerWin,OppFTA_PerWin,OppOR_PerWin,OppDR_PerWin,OppAst_PerWin,OppTO_PerWin,OppStl_PerWin,OppBlk_PerWin,OppPF_PerWin,Win_PerWin,FG_Percentage_PerWin,FG3_Percentage_PerWin,FT_Percentage_PerWin,Ast_TO_Ratio_PerWin,OppFG_Percentage_PerWin,OppFG3_Percentage_PerWin,OppFT_Percentage_PerWin,OppAst_TO_Ratio_PerWin,Poss_PerWin,OppPoss_PerWin,OffEff_PerWin,DefEff_PerWin,OppOffEff_PerWin,OppDefEff_PerWin,Score_Diff_PerWin,FGM_Diff_PerWin,FGA_Diff_PerWin,FGM3_Diff_PerWin,FGA3_Diff_PerWin,FTM_Diff_PerWin,FTA_Diff_PerWin,OR_Diff_PerWin,DR_Diff_PerWin,Ast_Diff_PerWin,TO_Diff_PerWin,Stl_Diff_PerWin,Blk_Diff_PerWin,PF_Diff_PerWin,Poss_Diff_PerWin,OffEff_Diff_PerWin,DefEff_Diff_PerWin,Score_Diff_Pct_PerWin,FGM_Diff_Pct_PerWin,FGA_Diff_Pct_PerWin,FGM3_Diff_Pct_PerWin,FGA3_Diff_Pct_PerWin,FTM_Diff_Pct_PerWin,FTA_Diff_Pct_PerWin,OR_Diff_Pct_PerWin,DR_Diff_Pct_PerWin,Ast_Diff_Pct_PerWin,TO_Diff_Pct_PerWin,Stl_Diff_Pct_PerWin,Blk_Diff_Pct_PerWin,PF_Diff_Pct_PerWin,Poss_Diff_Pct_PerWin,OffEff_Diff_Pct_PerWin,DefEff_Diff_Pct_PerWin,Score_Diff_Opp_PerWin,FGM_Diff_Opp_PerWin,FGA_Diff_Opp_PerWin,FGM3_Diff_Opp_PerWin,FGA3_Diff_Opp_PerWin,FTM_Diff_Opp_PerWin,FTA_Diff_Opp_PerWin,OR_Diff_Opp_PerWin,DR_Diff_Opp_PerWin,Ast_Diff_Opp_PerWin,TO_Diff_Opp_PerWin,Stl_Diff_Opp_PerWin,Blk_Diff_Opp_PerWin,PF_Diff_Opp_PerWin,Poss_Diff_Opp_PerWin,OffEff_Diff_Opp_PerWin,DefEff_Diff_Opp_PerWin,Score_Diff_Pct_Opp_PerWin,FGM_Diff_Pct_Opp_PerWin,FGA_Diff_Pc